<a href="https://colab.research.google.com/github/LoPA607/SOC-2024/blob/main/WEEK%201/Assignment_1(both_implementation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import math

In [ ]:
bc=datasets.load_breast_cancer()
X,y=bc.data, bc.target

In [ ]:
class load_breast_cancer(Dataset):

    def __init__(self):
        xy = np.loadtxt(bc, delimiter=',', dtype=np.float32, skiprows=1)
        self.x = torch.from_numpy(xy[:, 1:])
        self.y= torch.from_numpy(xy[:, [0]])
        self.n_samples = xy.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]


    def __len__(self):
        return self.n_samples


In [ ]:
dataset=bc


In [ ]:
n_samples,n_features=X.shape
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.9859, random_state=1234)

In [ ]:
dataloader=DataLoader(list(zip(X_train,y_train)), batch_size=4, shuffle=True)


In [ ]:
X_train.shape

(8, 30)

In [ ]:
len(dataset)

8

In [ ]:
print(dataloader.batch_size)

4


In [ ]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
X_train=torch.from_numpy(X_train.astype(np.float32))
X_test=torch.from_numpy(X_test.astype(np.float32))
y_train=torch.from_numpy(y_train.astype(np.float32))
y_test=torch.from_numpy(y_test.astype(np.float32))

In [ ]:
y_train=y_train.view(y_train.shape[0],1)
y_test=y_test.view(y_test.shape[0],1)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted


In [ ]:
model=LogisticRegression(n_features)


In [ ]:
learning_rate=0.01
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)
num_epochs=100
total_samples=len(dataset)
n_iterations=math.ceil(total_samples/4)
for epoch in range(num_epochs):
  for i, (inputs,labels) in enumerate(dataloader):
      if (i + 1) % 5 == 0:
        print(f'epoch{epoch + 1}/{num_epochs}, step {i + 1}/{n_iterations}, input {inputs.shape}')
      optimizer.zero_grad()
      y_predicted = model(X_train)
      loss = criterion(y_predicted, y_train)
      loss.backward()
      optimizer.step()
  if (epoch + 1) % 10 == 0:
        print(f'epoch:{epoch + 1}, loss={loss.item():.4f}')

epoch:10, loss=0.0434
epoch:20, loss=0.0142
epoch:30, loss=0.0089
epoch:40, loss=0.0066
epoch:50, loss=0.0053
epoch:60, loss=0.0043
epoch:70, loss=0.0036
epoch:80, loss=0.0031
epoch:90, loss=0.0027
epoch:100, loss=0.0023


In [ ]:
with torch.no_grad():
  y_predicted=model(X_test)
  y_predicted_cls=y_predicted.round()
  acc=y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
  print(f'accuracy={acc:.4f}')

accuracy=0.8378
